In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from tcn import TCN
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Concatenate, Attention, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings

warnings.filterwarnings('ignore')


2024-03-11 20:45:13.343563: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-11 20:45:13.387299: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-11 20:45:13.387331: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-11 20:45:13.387354: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-11 20:45:13.395506: I tensorflow/core/platform/cpu_feature_g

In [2]:
file_path = "myData2.parquet"
df = pd.read_parquet(file_path)


In [3]:
selected_features = ['timestamp_seconds', 
                     'node_memory_Percpu_bytes', 
                     'node_context_switches_total', 
                     'surfsara_power_usage', 
                     'node_netstat_Tcp_InSegs', 
                     'node_netstat_Tcp_OutSegs', 
                     'node_network_transmit_packets_total-sum', 
                     'node_filesystem_size_bytes-sum', 
                     'node_filesystem_files-sum', 
                     'node_memory_MemFree_bytes', 
                     'node_netstat_Tcp_InErrs']
# FixMe

In [4]:
# Define time intervals
time_intervals = {'minute': '1T', 'hour': '1H', 'day': '1D'}

# Set sequence length
sequence_length = 30

# Number of time steps to predict into the future
prediction_steps = 7

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


In [5]:
# Function to prepare data
def prepare_data(data, time_interval):
    data.set_index('timestamp', inplace=True)
    data_resampled = data.resample(time_interval).sum()
    
    target_mean = data_resampled['target'].mean()
    target_std = data_resampled['target'].std()
    data_resampled['target'] = (data_resampled['target'] - target_mean) / target_std
    
    target_min = data_resampled['target'].min()
    target_max = data_resampled['target'].max()
    print("Minimum value of target variable:", target_min)
    print("Maximum value of target variable:", target_max)
    
    return data_resampled

# Function to create sequences
def create_sequences(data, sequence_length):
    sequences, targets = [], []
    for i in range(len(data) - sequence_length):
        seq = data.iloc[i:i+sequence_length].values
        target = data.iloc[i+sequence_length]['target']
        sequences.append(seq)
        targets.append(target)
    return np.array(sequences), np.array(targets)


In [6]:
# Function to create a hybrid model with attention mechanism
def create_attention_hybrid_model(lstm_model, tcn_model):
    lstm_input = lstm_model.input
    tcn_input = tcn_model.input

    # Get the output layers of both models
    lstm_output = lstm_model.layers[-1].output
    tcn_output = tcn_model.layers[-1].output

    # Use Attention mechanism to combine outputs
    attention = Attention()([lstm_output, tcn_output])
    merged = Concatenate()([lstm_output, tcn_output, attention])

    # Add a dense layer for the final prediction
    merged = Dense(1, activation='sigmoid')(merged)

    # Create the ensemble model
    ensemble_model = Model(inputs=[lstm_input, tcn_input], outputs=merged)

    # Compile the model
    ensemble_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

    return ensemble_model


In [7]:
# Function to make predictions on new data for the hybrid model
def predict_future_failures_hybrid(model, input_data_lstm, input_data_tcn, sequence_length, prediction_steps):
    predictions = []

    for _ in range(prediction_steps):
        # Make predictions for the next time step using both LSTM and TCN models
        prediction = model.predict([input_data_lstm.reshape(1, sequence_length, input_data_lstm.shape[1]),
                                    input_data_tcn.reshape(1, sequence_length, input_data_tcn.shape[1])])
        predictions.append(prediction[0, 0])

        # Shift the input data by one time step and append the new prediction
        input_data_lstm = np.roll(input_data_lstm, shift=-1, axis=0)
        input_data_lstm[-1, -1] = prediction[0, 0]

        input_data_tcn = np.roll(input_data_tcn, shift=-1, axis=0)
        input_data_tcn[-1, -1] = prediction[0, 0]

    return predictions


# 30 days -> 7 days

In [8]:
# Extract relevant columns
df_selected = df[['timestamp', 'state'] + selected_features].copy()

# Encode the target variable 'state' to binary (0 for "COMPLETED", 1 otherwise)
df_selected['target'] = (df_selected['state'] != 'COMPLETED').astype(int)

# Drop the original 'state' column
df_selected.drop('state', axis=1, inplace=True)

# Normalize selected features
scaler = MinMaxScaler()
df_selected[selected_features] = scaler.fit_transform(df_selected[selected_features])


In [9]:
# Prepare data with daily intervals
data_day = prepare_data(df_selected, time_intervals['day'])

# Create sequences and targets
sequences_day, targets_day = create_sequences(data_day, sequence_length)

# Define the index to split the data
split_index_day = int(len(sequences_day) * 0.7)  # Use 70% of the data for training

# Split the data into training and testing sets
X_train_day, X_test_day = sequences_day[:split_index_day], sequences_day[split_index_day:]
y_train_day, y_test_day = targets_day[:split_index_day], targets_day[split_index_day:]

# Build the LSTM model
lstm_model_day = Sequential()
lstm_model_day.add(LSTM(50, input_shape=(X_train_day.shape[1], X_train_day.shape[2])))
lstm_model_day.add(Dense(1, activation='sigmoid'))
lstm_model_day.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

# Train the model
lstm_model_day.fit(X_train_day, y_train_day, epochs=10, batch_size=16, validation_split=0.15)

# Build the TCN model
tcn_model_day = Sequential([
    TCN(input_shape=(sequence_length, X_train_day.shape[2])),
    Dense(1, activation='sigmoid')
])
tcn_model_day.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

# Train the model
tcn_model_day.fit(X_train_day, y_train_day, epochs=20, batch_size=32, validation_split=0.1)

# Create the hybrid model
hybrid_model_attention_day = create_attention_hybrid_model(lstm_model_day, tcn_model_day)

# Train the hybrid model with both LSTM and TCN data
history_day = hybrid_model_attention_day.fit([X_train_day, X_train_day], y_train_day, epochs=20, batch_size=32, validation_split=0.1)


Minimum value of target variable: -0.7362924419461765
Maximum value of target variable: 5.322951443655186
Epoch 1/10
4/4 [==============================] - 3s 196ms/step - loss: 0.7782 - mean_squared_error: 0.7782 - val_loss: 0.9101 - val_mean_squared_error: 0.9101
Epoch 2/10
4/4 [==============================] - 0s 35ms/step - loss: 0.7660 - mean_squared_error: 0.7660 - val_loss: 0.8969 - val_mean_squared_error: 0.8969
Epoch 3/10
4/4 [==============================] - 0s 35ms/step - loss: 0.7728 - mean_squared_error: 0.7728 - val_loss: 0.8794 - val_mean_squared_error: 0.8794
Epoch 4/10
4/4 [==============================] - 0s 35ms/step - loss: 0.7787 - mean_squared_error: 0.7787 - val_loss: 0.8702 - val_mean_squared_error: 0.8702
Epoch 5/10
4/4 [==============================] - 0s 36ms/step - loss: 0.7784 - mean_squared_error: 0.7784 - val_loss: 0.8622 - val_mean_squared_error: 0.8622
Epoch 6/10
4/4 [==============================] - 0s 35ms/step - loss: 0.7753 - mean_squared_error

## Performance_Day

In [10]:
# Evaluate the model using Mean Squared Error
mse_day = hybrid_model_attention_day.evaluate([X_test_day, X_test_day], y_test_day, verbose=0)[1]
print(f'Mean Squared Error: {mse_day:.4f}\n')


Mean Squared Error: 2.0995



In [13]:
# Evaluate model on test data
y_pred_day = hybrid_model_attention_day.predict(X_test_day, X_test_day)



ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
# Calculate Root Mean Squared Error
rmse_day = mean_squared_error(y_test_day, y_pred_day, squared=False)
print(f'Root Mean Squared Error (RMSE): {rmse_day:.4f}\n')

# Calculate Mean Absolute Error
mae_day = mean_absolute_error(y_test_day, y_pred_day)
print(f'Mean Abosolute Error: {mae_day:.4f}\n')

# Calculate R-squared
r2_day = r2_score(y_test_day, y_pred_day)
print(f'R-squared (R2): {r2_day:.4f}\n')

### Training and Validation Loss Plot

In [ ]:
# Plot training and validation loss
plt.plot(history_day.history['loss'], label='Training Loss')
plt.plot(history_day.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()


### Predicted Failures vs. True Failures Plot

In [ ]:
target_min_day = -0.7362924419461765
target_max_day = 5.322951443655186

In [ ]:
index_day = 0  # Can be any valid starting point

input_data_day = X_test_day[index_day]

# Make predictions
predicted_failures_day = predict_future_failures_hybrid(lstm_model_day, input_data_day, sequence_length, prediction_steps)

# # Denormalize the predicted failures 
# predicted_failures_denormalized_day = np.array(predicted_failures_day) * (target_max_day - target_min_day) + target_min_day

# Get the true failures for the specified number of days
true_failures_day = y_test_day[index_day:index_day + prediction_steps]

# Print the predicted failures
print("Predicted failures for the next 7 days:")
print(predicted_failures_day)

# Evaluate the predictions using Mean Squared Error
mse_predictions = np.mean((predicted_failures_day - true_failures_day)**2)
print(f'\nMean Squared Error for Predictions: {mse_predictions:.4f}\n')

# Evaluate the predictions using Mean Absolute Error
mae_predictions = np.mean(np.abs(predicted_failures_day - true_failures_day))
print(f'Mean Absolute Error for Predictions: {mae_predictions:.4f}')

# Plot predicted failures vs. true failures
plt.figure(figsize=(10, 6))
plt.plot(predicted_failures_day, label='Predicted Failures', marker='o')
plt.plot(true_failures_day, label='True Failures', marker='x')
plt.xlabel('Days')
plt.ylabel('Number of Failures')
plt.title('Predicted Failures vs. True Failures')
plt.legend()
plt.grid(True)
plt.show()


Model Mean Squared Error: 2.3448

1/1 [==============================] - 0s 25ms/step
Predicted Failures for the Next 7 Time Steps:
[0.5312288, 0.49342206, 0.49473184, 0.5320658, 0.53384113, 0.5329574, 0.5334162]

Mean Squared Error for Predictions: 4.6117

Mean Absolute Error for Predictions: 1.7128


# 30 hours -> 7 hours

In [ ]:
# Extract relevant columns
df_selected = df[['timestamp', 'state'] + selected_features].copy()

# Encode the target variable 'state' to binary (0 for "COMPLETED", 1 otherwise)
df_selected['target'] = (df_selected['state'] != 'COMPLETED').astype(int)

# Drop the original 'state' column
df_selected.drop('state', axis=1, inplace=True)

# Normalize selected features
scaler = MinMaxScaler()
df_selected[selected_features] = scaler.fit_transform(df_selected[selected_features])


In [ ]:
# Prepare data with hourly intervals
data_hour = prepare_data(df_selected, time_intervals['hour'])

# Create sequences and targets
sequences_hour, targets_hour = create_sequences(data_hour, sequence_length)

# Define the index to split the data
split_index_hour = int(len(sequences_hour) * 0.7)  # Use 70% of the data for training

# Split the data into training and testing sets
X_train_hour, X_test_hour = sequences_hour[:split_index_hour], sequences_hour[split_index_hour:]
y_train_hour, y_test_hour = targets_hour[:split_index_hour], targets_hour[split_index_hour:]

# Build the LSTM model
lstm_model_hour = Sequential()
lstm_model_hour.add(LSTM(50, input_shape=(X_train_hour.shape[1], X_train_hour.shape[2])))
lstm_model_hour.add(Dense(1, activation='sigmoid'))
lstm_model_hour.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

# Train the model
lstm_model_hour.fit(X_train_hour, y_train_hour, epochs=10, batch_size=16, validation_split=0.15)

# Build the TCN model
tcn_model_hour = Sequential([
    TCN(input_shape=(sequence_length, X_train_hour.shape[2])),
    Dense(1, activation='sigmoid')
])
tcn_model_hour.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

# Train the model
tcn_model_hour.fit(X_train_hour, y_train_hour, epochs=20, batch_size=32, validation_split=0.1)

# Create the hybrid model
hybrid_model_hour_attention = create_attention_hybrid_model(lstm_model_hour, tcn_model_hour)

# Train the hybrid model with both LSTM and TCN data
hybrid_model_hour_attention.fit([X_train_hour, X_train_hour], y_train_hour, epochs=20, batch_size=32, validation_split=0.1)


KeyError: "None of ['timestamp'] are in the columns"

## Performance_Hour

In [ ]:
# Evaluate the model using Mean Squared Error
mse_hour = lstm_model_hour.evaluate(X_test_hour, y_test_hour, verbose=0)[1]
print(f'Mean Squared Error: {mse_hour:.4f}\n')


In [ ]:
# Evaluate model on test data
y_pred_hour = lstm_model_hour.predict(X_test_hour)

# Calculate Root Mean Squared Error
rmse_hour = mean_squared_error(y_test_hour, y_pred_hour, squared=False)
print(f'Root Mean Squared Error (RMSE): {rmse_hour:.4f}\n')

# Calculate Mean Absolute Error
mae_hour = mean_absolute_error(y_test_hour, y_pred_hour)
print(f'Mean Abosolute Error: {mae_hour:.4f}\n')

# Calculate R-squared
r2_hour = r2_score(y_test_hour, y_pred_hour)
print(f'R-squared (R2): {r2_hour:.4f}\n')


In [ ]:
# Evaluate the model using Mean Squared Error
mse_hour_attention = hybrid_model_hour_attention.evaluate([X_test_hour, X_test_hour], y_test_hour, verbose=0)[1]
print(f'Model Mean Squared Error: {mse_hour_attention:.4f}\n')

# Make predictions with the hybrid model
predicted_failures_hybrid_attention = predict_future_failures_hybrid(hybrid_model_hour_attention, input_data_lstm, input_data_tcn, sequence_length, prediction_steps_hybrid)

# Print the predicted failures
print("Predicted Failures for the Next 7 Time Steps (Hybrid):")
print(predicted_failures_hybrid_attention)

# Evaluate the predictions using Mean Squared Error
mse_predictions = np.mean((predicted_failures_hybrid_attention - y_test_hour[3:3+prediction_steps])**2)
print(f'\nMean Squared Error for Predictions: {mse_predictions:.4f}\n')

# Evaluate the predictions using Mean Absolute Error
mae_predictions_hybrid_attention_hour = np.mean(np.abs(predicted_failures_hybrid_attention - y_test_hour[3:3+prediction_steps]))
print(f'Mean Absolute Error for Predictions: {mae_predictions_hybrid_attention_hour:.4f}\n')


Model Mean Squared Error: 0.1999

1/1 [==============================] - 0s 28ms/step
Predicted Failures for the Next 7 Time Steps (Hybrid):
[0.4002875, 0.4002881, 0.4002926, 0.4003259, 0.40057176, 0.4023942, 0.41613513]

Mean Squared Error for Predictions: 0.2193

Mean Absolute Error for Predictions: 0.4598



# Minute

In [ ]:
# Prepare data
data_minute = prepare_data(df_selected, time_intervals['minute'])

# Create sequences and targets
sequences_minute, targets_minute = create_sequences(data_minute, sequence_length)

# Split the data into training and testing sets
X_train_minute, X_test_minute, y_train_minute, y_test_minute = train_test_split(sequences_minute, targets_minute, test_size=0.2, random_state=1)

# Build the LSTM model
lstm_model_minute = Sequential()
lstm_model_minute.add(LSTM(50, input_shape=(X_train_minute.shape[1], X_train_minute.shape[2])))
lstm_model_minute.add(Dense(1, activation='sigmoid'))
lstm_model_minute.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

# Train the model
lstm_model_minute.fit(X_train_minute, y_train_minute, epochs=10, batch_size=16, validation_split=0.15)

# Build the TCN model
tcn_model_minute = Sequential([
    TCN(input_shape=(sequence_length, X_train_minute.shape[2])),
    Dense(1, activation='sigmoid')
])

# Compile the model
tcn_model_minute.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

# Train the model
tcn_model_minute.fit(X_train_minute, y_train_minute, epochs=20, batch_size=32, validation_split=0.1)

# Select a starting point for predictions
input_data_lstm_hybrid = X_test_minute[3]
input_data_tcn_hybrid = X_test_minute[3]

# Number of time steps to predict into the future
prediction_steps = 7

# Create the hybrid model
hybrid_model_minute_attention = create_attention_hybrid_model(lstm_model_minute, tcn_model_minute)

# Train the hybrid model with both LSTM and TCN data
hybrid_model_minute_attention.fit([X_train_minute, X_train_minute], y_train_minute, epochs=10, batch_size=32, validation_split=0.1)


Epoch 1/10
7033/7033 [==============================] - 153s 21ms/step - loss: 0.0044 - mean_squared_error: 0.0044 - val_loss: 0.0017 - val_mean_squared_error: 0.0017
Epoch 2/10
7033/7033 [==============================] - 150s 21ms/step - loss: 0.0014 - mean_squared_error: 0.0014 - val_loss: 0.0015 - val_mean_squared_error: 0.0015
Epoch 3/10
7033/7033 [==============================] - 137s 19ms/step - loss: 0.0013 - mean_squared_error: 0.0013 - val_loss: 0.0012 - val_mean_squared_error: 0.0012
Epoch 4/10
7033/7033 [==============================] - 127s 18ms/step - loss: 0.0013 - mean_squared_error: 0.0013 - val_loss: 0.0013 - val_mean_squared_error: 0.0013
Epoch 5/10
7033/7033 [==============================] - 132s 19ms/step - loss: 0.0013 - mean_squared_error: 0.0013 - val_loss: 0.0014 - val_mean_squared_error: 0.0014
Epoch 6/10
7033/7033 [==============================] - 120s 17ms/step - loss: 0.0012 - mean_squared_error: 0.0012 - val_loss: 0.0012 - val_mean_squared_error: 0.001

In [ ]:
# Evaluate the model using Mean Squared Error
mse_minute_attention = hybrid_model_minute_attention.evaluate([X_test_minute, X_test_minute], y_test_minute, verbose=0)[1]
print(f'Model Mean Squared Error: {mse_minute_attention:.4f}\n')

# Make predictions with the hybrid model
predicted_failures_hybrid_attention = predict_future_failures_hybrid(hybrid_model_minute_attention, input_data_lstm_hybrid, input_data_tcn_hybrid, sequence_length, prediction_steps)

# Print the predicted failures
print("Predicted Failures for the Next 7 Time Steps (Hybrid):")
print(predicted_failures_hybrid_attention)

# Evaluate the predictions using Mean Squared Error
mse_predictions = np.mean((predicted_failures_hybrid_attention - y_test_minute[3:3+prediction_steps])**2)
print(f'\nMean Squared Error for Predictions: {mse_predictions:.4f}\n')

# Evaluate the predictions using Mean Absolute Error
mae_predictions_hybrid_attention_minute = np.mean(np.abs(predicted_failures_hybrid_attention - y_test_minute[3:3+prediction_steps]))
print(f'Mean Absolute Error for Predictions: {mae_predictions_hybrid_attention_minute:.4f}\n')


Model Mean Squared Error: 0.0013

1/1 [==============================] - 0s 24ms/step
Predicted Failures for the Next 7 Time Steps (Hybrid):
[0.003039969, 0.0030414464, 0.0030468209, 0.0030486726, 0.0030448635, 0.0030444176, 0.003047832]

Mean Squared Error for Predictions: 0.4260

Mean Absolute Error for Predictions: 0.4290

